# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    precision = [1 if p in gold else 0 for p in pred]
    precision = sum(precision)/(len(precision))

    recall = [1 if g in pred else 0 for g in gold]
    recall = sum(recall)/(len(recall))

    print('precision: {0}%'.format(round(precision*100)) )
    print('recall: {0}%'.format(round(recall*100)) )
    print('F1: {0}%'.format(round(200*((precision*recall)/(precision+recall)))) )

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

precision: 60%
recall: 100%
F1: 75%


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    for row in df.itertuples():
        yield (row[1], row[3], row[4])

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [8]:
spans_dev_gold = list(gold_spans(df_dev))
print(len(spans_dev_gold))
print(spans_dev_gold[0])
print(spans_dev_gold[5916])

5917
('0946-000', 2, 3)
('1161-010', 1, 3)


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [9]:
import spacy

def ne_pred(df):
    for row in df.itertuples():
        senid = row[1]
        sen = row[2]
        doc = nlp(sen)
        for ent in doc.ents:
            yield senid, ent.start, ent.end

            res = set(ne_pred(df_dev))
spans_dev_gold = set(gold_spans(df_dev))
evaluation_report(spans_dev_gold,res)

precision: 53%
recall: 69%
F1: 60%


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [10]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [11]:
df_500 = df_dev[0:500]
res = set(ne_pred(df_500))
spans_dev_gold = set(gold_spans(df_500))
error_report(df_dev,spans_dev_gold,res)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
  FP: four
  FP: 39
  FP: 38
  FP: two days
  FP: Friday
  FN: Somerset
Sentence: Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .
  FN: Nottinghamshire
Sentence: After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
  FP: 94
  FP: three
  FP: 83
  FP: 296
  FP: the opening morning
  FP: first
  FP: 83
  FN: Leicestershire
Sentence: Trailing by 213 , Somerset got a solid start to th

*TODO: Write a short text that summarises the errors that you observed*
Many of the errors have to do with either numbers or dates, which should be easy to filter out

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [12]:
def filtered_pred(df):
    for row in df.itertuples():
        senid = row[1]
        sen = row[2]
        doc = nlp(sen)
        for ent in doc.ents:
            if(ent.label_!="DATE" and ent.label_!="MONEY" and ent.label_!="QUANTITY" and ent.label_!="CARDINAL" and ent.label_!="ORDINAL"):
                yield senid, ent.start, ent.end
df_500 = df_dev[0:500]
res = set(filtered_pred(df_500))
spans_dev_gold = set(gold_spans(df_500))
#error_report(df_dev,spans_dev_gold,res)
evaluation_report(spans_dev_gold,res)

precision: 81%
recall: 64%
F1: 72%


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [13]:
restemp = []
col = ["sentence_id", "sentence", "beg", "end", "label"]
for i in res:
    senid, beg, end = i
    label = "--NME--"
    sen = df_dev[df_dev["sentence_id"]==senid].iloc[0]["sentence"]
    restemp.append((senid, sen, beg, end, label))

res_df = pd.DataFrame(restemp, columns=col)
res_df.head()

,sentence_id,sentence,beg,end,label
0,0957-020,"Add Men 's singles , second round 1 - Pete Sam...",12,13,--NME--
1,0949-004,Adams and Platt are both injured and will miss...,9,10,--NME--
2,0946-012,Australian Tom Moody took six for 82 but Chris...,20,21,--NME--
3,0954-002,Quarter-final results in the Hong Kong Open on...,27,28,--NME--
4,0946-003,"Their stay on top , though , may be short-live...",17,18,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [14]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    for row in df.itertuples():
        yield (row[1], row[3], row[4], row[5])

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [15]:
def baseline(df):
    for row in df.itertuples():
        split = row[2].split(" ")[row[3]:row[4]]
        joined = "_".join(split)
        yield (row[1], row[3], row[4], joined)

baseline_data = set(baseline(res_df))
gold_new = set(gold_mentions(df_500))
evaluation_report(gold_new,baseline_data)

precision: 28%
recall: 22%
F1: 25%


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [16]:
with bz2.open('kb.tsv.bz2', 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [17]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [18]:
def mostprob(df):
    for row in df.itertuples():
        split = row[2].split(" ")[row[3]:row[4]]
        joined = "_".join(split)
        entity = df_kb[df_kb["mention"] == joined]
        if(len(entity)>0):
            pred_label = entity.iloc[0].entity
        else:
            pred_label = "--NME--"
        yield (row[1], row[3], row[4], pred_label)

mostprob_data = set(mostprob(res_df))
evaluation_report(gold_new,mostprob_data)

precision: 40%
recall: 32%
F1: 36%


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [19]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [20]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [21]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


# Set up a dictionary that contains a seperate Naive Bayes classifier,
# for each mention 𝑚 that can refer to more than one entity 𝑒,
classifier_dict = {}

for mention in set(df_contexts['mention']):
    entities = set(df_contexts.entity[df_contexts.mention == mention])
    if len(entities) > 1:
        contexts, y = np.array([]), []
        for ent in entities:
            c = df_contexts.context[df_contexts.entity == ent]
            contexts = np.append(contexts, np.array(c))
            y += [ent for i in range(len(c))]

        x = contexts.flatten()
        
    probabilities = df_kb.loc[df_kb.mention == mention].sort_values(by=['entity']).prob
    pipe = Pipeline([('count_vectorizer', CountVectorizer()), ('classifier', MultinomialNB(class_prior=probabilities.values))])
    pipe.fit(x, y)
    classifier_dict[mention] = pipe


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

In [25]:
def most_prob_nb(df):
    for row in df.itertuples():
        split_ent = row[2].split(" ")[row[3]:row[4]]
        joined_ent = "_".join(split_ent)
        entity = df_kb[df_kb["mention"] == joined_ent]
            
        if (len(entity) == 1):
            pred_label = entity.iloc[0].entity
        elif (len(entity) > 1):
            split_sent = row[2].split(" ")
            left, right = split_sent[:row[3]][-5:], split_sent[row[4]:][:5]
            context = " ".join(left+right)
            pred_label = classifier_dict[joined_ent].predict([context])[0]
        else:
            pred_label = "--NME--"
        
        yield (row[1], row[3], row[4], pred_label)

most_prob_data = set(most_prob_nb(res_df))
evaluation_report(gold_new, most_prob_data)   

precision: 43%
recall: 34%
F1: 38%


Reflection questions
The following reflection questions are questions that you could be asked in the oral exam. Try to answer each of them in the form of a short text and enter it in the cell below. You will get feedback on your answers from your lab assistant.

RQ 5.1: In Problem 3, you did an error analysis on the task of recognizing text spans mentioning named entities. Summarize your results. Pick one type of error that you observed. How could you improve the model’s performance on this type of error? What resources (such as domain knowledge, data, compute) would you need to implement this improvement?
>In the results many of the errors where different kinds of numbers, dates or ordinals. To improve the performance and eliminate this error, we can use the filtering methods used in this lab. We can also use domain specific data to fine tune this model and retrain it to be more useful on our data.

RQ 5.2: Thinking back about Problem 6, explain what the word context refers to in the task addressed there, and how context can help to disambiguate between different entities. Suggest other types of context that you could use for disambiguation.
>The word context in problem 6 refers to the five tokens before and after the given entity. A mention can refer to many different entities, for example; 'Washington' can refer to the city, the state or even the president. By looking at the context and finding words like 'city', 'D.C', 'George' or 'state' it becomes possible to disambiguate the entity. It is in a sense using the idea that we can "know a word by the company it keeps".
Instead of looking at the semantic context it appears in, it is also possible to look at the syntactic context. For example, looking at what part of speech tag it is and what part of speech tags that surround it.

RQ 5.3: One type of entity mentions that we did not cover explicitly in this lab are pronouns. As an example, consider the sentence pair Ruth Bader Ginsburg was an American jurist. She served as an associate justice of the Supreme Court from 1993 until her death in 2020. What facts would you want to extract from this sentence pair? How do pronouns make fact extraction hard?
>The fact that Ruth Bader Ginsburg is the one who served as an associate justice. However, since she is replaced with the word 'she' in the second sentence, it becomes impossible to now who it is if we only look at one sentence at a time. To counter this, there is a need to keep track of several sentences at all times. But many times the sentences are independent from each other as well. Thus, it becomes difficult.

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>